In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import seaborn as sns
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('avx-frequency-license-bands-validation'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
variants = glob.glob(f'{experiment.path}/lic*')
variants

In [ ]:
def read_from_lic(folder: str) -> pd.DataFrame:
    """
    folder: The folder that contains the measurements for the different number of cores.
    """
    df = pd.DataFrame()

    for file in glob.glob(f'{folder}/*'):
        num_cores = int(Path(file).stem) + 1
        read_df = pd.read_csv(file)
        pd.read_csv
        for _index, row in read_df.iterrows():
            metric = row['metric']
            value = row['average']
            if (metric == 'perf-freq'):
                value *= num_cores
            df = pd.concat([df, pd.DataFrame([{'metric': metric, 'value': value, 'core_count': num_cores}])])

    return df

In [ ]:
data = pd.DataFrame()

for variant in variants:
    row = read_from_lic(variant)
    license = variant.split('/')[-1]
    row['license'] = license
    data = pd.concat([data, row])

data

In [ ]:
license_level_to_name = {
    'lic0': 'SSE',
    'lic1': 'AVX2',
    'lic2': 'AVX512',
    'lic3': 'AMX'
}

LIC_NAME = 'Frequency class'
CORE_COUNT = 'Number of active cores'
MAX_TURBO_FREQ = 'Maximum opportunistic turbo frequency [MHz]'
POWER_WATT = 'Power draw of package 0 [W]'

data[LIC_NAME] = data['license'].apply(lambda v: license_level_to_name[v])
data[CORE_COUNT] = data['core_count']

data[MAX_TURBO_FREQ] = data['value'] * 1000
data[POWER_WATT] = data['value']

In [ ]:
hue_order=['AMX', 'AVX512', 'AVX2', 'SSE']

In [ ]:
ax = sns.scatterplot(data=data[data['metric'] == 'perf-freq'], x=CORE_COUNT, y=MAX_TURBO_FREQ, hue=LIC_NAME, linewidth=1.5, marker='o', hue_order=hue_order)
plt.xticks(rotation=45)

buckets = [28,34,40,44,48,50,54,56]

for i in buckets:
    plt.axvline(x=i, color='black', dashes=[2], linewidth=1)

# ax.set_ylim(0, 4000)
ax.set_xlim(0, 58)
ax.set_xticks([0] + buckets)

# plt.xticks(rotation=45)
ax.tick_params(axis='x', pad=0)

Plotting.savefig(experiment, 'validate-avx-frequency-license-bands.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)